In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# --- Step 1: Download stock data ---
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "NVDA", "JPM", "UNH", "XOM"]
data = yf.download(tickers, start="2020-01-01", end="2025-01-01")["Close"]

# Inspect data
print(data.head())

In [3]:
# --- Step 2: Compute log returns ---
returns = np.log(data / data.shift(1)).dropna()


In [4]:
# --- Step 3: Portfolio Simulation ---
np.random.seed(42)
num_portfolios = 5000

results = np.zeros((3, num_portfolios))
weights_record = []

for i in range(num_portfolios):
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)
    weights_record.append(weights)

    portfolio_return = np.sum(returns.mean() * weights) * 252
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(returns.cov() * 252, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility

    results[0, i] = portfolio_return
    results[1, i] = portfolio_volatility
    results[2, i] = sharpe_ratio


In [ ]:
# --- Step 4: Efficient Frontier Plot ---
results_df = pd.DataFrame(results.T, columns=["Return", "Volatility", "Sharpe"])
plt.figure(figsize=(10,6))
plt.scatter(results_df["Volatility"], results_df["Return"], c=results_df["Sharpe"], cmap="viridis", s=10)
plt.colorbar(label="Sharpe Ratio")
plt.xlabel("Volatility (Std Dev)")
plt.ylabel("Expected Return")
plt.title("Efficient Frontier - Portfolio Optimization")

# Highlight max Sharpe portfolio
max_sharpe_idx = results_df["Sharpe"].idxmax()
plt.scatter(results_df.loc[max_sharpe_idx]["Volatility"],
            results_df.loc[max_sharpe_idx]["Return"],
            c="red", s=50, marker="*", label="Max Sharpe Ratio")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(returns.corr(), annot=True, cmap="coolwarm")
plt.title("Stock Correlation Heatmap")
plt.show()